In [ ]:
"""

while hotspots not empty

each hotspots has a number of men and women in group

we will simulate the starting at a hotspot h from H  

pick random from the group and assign to eligible shelter (row in dec matrix -> fimnd appropirate masking) 

if shelter_1 capcity not full:
update shhlter_1 
remove person from group
othetwise:
pick again 
"""

total_num_ppl = 1000
men_pct = 0.5 
women_pct = 1 - men_pct

# matrix with each row is [num men in hotpot, num women in hotpsot]
ppl_per_hotspot = []

for hotspot in range(top_16_hotspots.shape[0]):
    num_ppl = top_16_hotspots.iloc[hotspot]['low_income_score'] * total_num_ppl
    men_in_hotspot = men_pct * num_ppl
    women_in_hotspot = women_pct * num_ppl
    ppl_per_hotspot.append([men_in_hotspot, women_in_hotspot])
 
ppl_per_hotspot = np.array(ppl_per_hotspot)  # convert list of lists to array
ppl_per_hotspot = np.round(ppl_per_hotspot, 0)  # round to nearest integer
#ppl_per_hotspot

#we will now have capacities per shelter 

cap_per_shelter = []  # each row is for each shgtelr porgram contaisn [num men, num women]

for idx, row in unique_shelters.iterrows():
    # Determine shelter sector
    sector = row['type']  # 'Men only', 'Women only', 'Both'
    
    # Pick the capacity type
    if row['CAPACITY_TYPE'] == 'Bed Based Capacity':
        cap = row['CAPACITY_ACTUAL_BED']
    else:
        cap = row['CAPACITY_ACTUAL_ROOM']
    
    # Assign men/women based on sector
    if sector == 'Men only':
        men = cap
        women = 0
    elif sector == 'Women only':
        men = 0
        women = cap
    elif sector.lower() in ['both', 'mixed']:
        # Split evenly for 'Both' shelters
        men = cap / 2
        women = cap / 2
    else:
        men = 0
        women = 0  # fallback for unknown type
    
    # Append as a row
    cap_per_shelter.append([men, women])

# Convert to NumPy array
cap_per_shelter = np.array(cap_per_shelter)

eligibility_men = (cap_per_shelter[:, 0] > 0).astype(int)
eligibility_women = (cap_per_shelter[:, 1] > 0).astype(int)

# Save as CSV
pd.DataFrame(eligibility_men, columns=['eligible']).to_csv('eligibility_matrix_men_initial.csv', index=False)
pd.DataFrame(eligibility_women, columns=['eligible']).to_csv('eligibility_matrix_women_initial.csv', index=False)

# Optional: print to check
print("Initial eligibility mask for men:\n", eligibility_men)
print("Initial eligibility mask for women:\n", eligibility_women)

import numpy as np
import pandas as pd

def decision_policy(policy_type, distances, gender, ppl_per_hotspot, cap_per_shelter):
    num_shelters, num_hotspots = distances.shape
    allocation = np.zeros((num_shelters, num_hotspots))

    # Choose column index for men/women
    gender_idx = 0 if gender.lower() == 'men' else 1
    shelter_capacity = cap_per_shelter[:, gender_idx].copy()

    # Loop over hotspots
    # hotspot_order = np.random.permutation(num_hotspots)
    hotspot_order = np.argsort(-ppl_per_hotspot[:, gender_idx])

    for h in hotspot_order:
        remaining = int(ppl_per_hotspot[h, gender_idx])
        if remaining <= 0:
            continue

        # Distances to shelters for this hotspot
        dist_to_shelters = distances.iloc[:, h].values

        while remaining > 0:
            # Eligible shelters (still have capacity)
            eligible_shelters = np.where(shelter_capacity > 0)[0]
    
            if len(eligible_shelters) == 0:
                print(f"No more capacity for {gender} at hotspot {h}")
                break

            if policy_type.lower() == 'greedy':
                # Pick closest eligible shelter
                closest_idx = eligible_shelters[np.argmin(dist_to_shelters[eligible_shelters])]
            elif policy_type.lower() == 'random':
                # Pick random eligible shelter
                closest_idx = np.random.choice(eligible_shelters)
            else:
                raise ValueError("policy_type must be 'random' or 'greedy'")

            # Assign one person to shelter
            allocation[closest_idx, h] += 1
            shelter_capacity[closest_idx] -= 1
            remaining -= 1

    ppl_per_hotspot = pd.DataFrame(ppl_per_hotspot)
    ppl_per_hotspot.to_csv('ppl_per_hotspot.csv')

    return allocation

distance_matrix = pd.read_csv(r'distance_amtrix.csv', index_col=0)
top_16_hotspots = pd.DataFrame(top_16_hotspots)
top_16_hotspots.to_csv('top_16_hotspots.csv')

allocation_men = decision_policy('greedy', distance_matrix, 'Men', ppl_per_hotspot, cap_per_shelter)
allocation_women = decision_policy('greedy', distance_matrix, 'Women', ppl_per_hotspot, cap_per_shelter)

ran_men = decision_policy('random', distance_matrix, 'Men', ppl_per_hotspot, cap_per_shelter)
rand_women = decision_policy('random', distance_matrix, 'Women', ppl_per_hotspot, cap_per_shelter)

Initial eligibility mask for men:
 [1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 1 1 1 1
 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 0 1]
Initial eligibility mask for women:
 [0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 0 1 1 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0]


In [ ]:
distances_array = distance_matrix.to_numpy()

total_distance_traveled_men = np.sum(allocation_men * distances_array)
total_distance_traveled_women = np.sum(allocation_women * distances_array)

print(f"Total distance traveled by men (greedy): {total_distance_traveled_men}")
print(f"Total distance traveled by women (greedy): {total_distance_traveled_women}")


total_distance_rand_men = np.sum(ran_men * distances_array)
total_distance_rand_women = np.sum(rand_women * distances_array)

print(f"Total distance traveled by men (random): {total_distance_rand_men}")
print(f"Total distance traveled by women (random): {total_distance_rand_women}")

Total distance traveled by men (greedy): 1240.9450544836386
Total distance traveled by women (greedy): 1883.5458925581115
Total distance traveled by men (random): 4017.8081805267484
Total distance traveled by women (random): 4203.775222254153


In [ ]:
import numpy as np
import pandas as pd

def run_allocation_simulation(distances, top_k_hotspots, unique_shelters, total_num_ppl=1000,men_pct=0.679, policy_type='greedy'):

    women_pct = 1 - men_pct
    
    ppl_per_hotspot = []
    for hotspot in range(top_k_hotspots.shape[0]):
        num_ppl = top_k_hotspots.iloc[hotspot]['low_income_score'] * total_num_ppl
        men_in_hotspot = men_pct * num_ppl
        women_in_hotspot = women_pct * num_ppl
        ppl_per_hotspot.append([men_in_hotspot, women_in_hotspot])
    ppl_per_hotspot = np.array(ppl_per_hotspot)
    ppl_per_hotspot = np.round(ppl_per_hotspot, 0)
    
    cap_per_shelter = []
    for idx, row in unique_shelters.iterrows():
        sector = row['type']
        cap = row['CAPACITY_ACTUAL_BED'] if row['CAPACITY_TYPE']=='Bed Based Capacity' else row['CAPACITY_ACTUAL_ROOM']
        if sector == 'Men only':
            men = cap
            women = 0
        elif sector == 'Women only':
            men = 0
            women = cap
        elif sector.lower() in ['both','mixed']:
            men = cap / 2
            women = cap / 2
        else:
            men = 0
            women = 0
        cap_per_shelter.append([men, women])
    cap_per_shelter = np.array(cap_per_shelter)
    
    def decision_policy(policy_type, distances, gender, ppl_per_hotspot, cap_per_shelter):
        num_shelters, num_hotspots = distances.shape
        allocation = np.zeros((num_shelters, num_hotspots))
        gender_idx = 0 if gender.lower()=='men' else 1
        shelter_capacity = cap_per_shelter[:, gender_idx].copy()
        
        # hotspot_order = np.random.permutation(num_hotspots)  # shuffle hotspots
        hotspot_order = np.argsort(-ppl_per_hotspot[:, gender_idx])  # sorts frim hghest capcity for gender to lowest


        for h in hotspot_order:
            remaining = int(ppl_per_hotspot[h, gender_idx])
            if remaining <= 0:
                continue
            dist_to_shelters = distances.iloc[:, h].values
            while remaining > 0:
                eligible_shelters = np.where(shelter_capacity>0)[0]
                if len(eligible_shelters)==0:
                    break
                if policy_type.lower()=='greedy':
                    closest_idx = eligible_shelters[np.argmin(dist_to_shelters[eligible_shelters])]
                elif policy_type.lower()=='random':
                    closest_idx = np.random.choice(eligible_shelters)
                else:
                    raise ValueError("policy_type must be 'greedy' or 'random'")
                allocation[closest_idx, h] += 1
                shelter_capacity[closest_idx] -= 1
                remaining -= 1
        return allocation
    
    # Run allocation
    allocation_men = decision_policy(policy_type, distances, 'Men', ppl_per_hotspot, cap_per_shelter)
    allocation_women = decision_policy(policy_type, distances, 'Women', ppl_per_hotspot, cap_per_shelter)
    
    distances_array = distances.to_numpy()
    total_distance_men = np.sum(allocation_men * distances_array)
    total_distance_women = np.sum(allocation_women * distances_array)
    
    return allocation_men, allocation_women, total_distance_men, total_distance_women

In [ ]:
alloc_men, alloc_women, dist_men, dist_women = run_allocation_simulation(distance_matrix, top_16_hotspots,unique_shelters,total_num_ppl=1000,men_pct=0.5,policy_type='greedy')

print("Total distance traveled by men:", dist_men)
print("Total distance traveled by women:", dist_women)


Total distance traveled by men: 1240.9450544836386
Total distance traveled by women: 1883.5458925581115


In [ ]:

num_sims_greedy = 100

alloc_men_total_greedy = np.zeros((unique_shelters.shape[0], top_16_hotspots.shape[0]))
alloc_women_total_greedy = np.zeros((unique_shelters.shape[0], top_16_hotspots.shape[0]))
dist_men_total_greedy = 0
dist_women_total_greedy = 0

for i in range(num_sims_greedy):
    alloc_men, alloc_women, dist_men, dist_women = run_allocation_simulation(
        distance_matrix, top_16_hotspots, unique_shelters,
        total_num_ppl=8000, men_pct=0.5, policy_type='greedy'
    )

    alloc_men_total_greedy += alloc_men
    alloc_women_total_greedy += alloc_women
    dist_men_total_greedy += dist_men
    dist_women_total_greedy += dist_women

alloc_men_avg_greedy = alloc_men_total_greedy / num_sims_greedy
alloc_women_avg_greedy = alloc_women_total_greedy / num_sims_greedy
dist_men_avg_greedy = dist_men_total_greedy / num_sims_greedy
dist_women_avg_greedy = dist_women_total_greedy / num_sims_greedy

print(f"Average total distance (greedy men):   {dist_men_avg_greedy}")
print(f"Average total distance (greedy women): {dist_women_avg_greedy}")


num_sims_random = 100

alloc_men_total_random = np.zeros((unique_shelters.shape[0], top_16_hotspots.shape[0]))
alloc_women_total_random = np.zeros((unique_shelters.shape[0], top_16_hotspots.shape[0]))
dist_men_total_random = 0
dist_women_total_random = 0

for i in range(num_sims_random):
    alloc_men, alloc_women, dist_men, dist_women = run_allocation_simulation(
        distance_matrix, top_16_hotspots, unique_shelters,
        total_num_ppl=8000, men_pct=0.5, policy_type='random'
    )

    alloc_men_total_random += alloc_men
    alloc_women_total_random += alloc_women
    dist_men_total_random += dist_men
    dist_women_total_random += dist_women

# Averages for random
alloc_men_avg_random = alloc_men_total_random / num_sims_random
alloc_women_avg_random = alloc_women_total_random / num_sims_random
dist_men_avg_random = dist_men_total_random / num_sims_random
dist_women_avg_random = dist_women_total_random / num_sims_random

print(f"Average total distance (random men):   {dist_men_avg_random}")
print(f"Average total distance (random women): {dist_women_avg_random}")


Average total distance (greedy men):   9641.580302513668
Average total distance (greedy women): 6202.919225908233
Average total distance (random men):   13714.744395266644
Average total distance (random women): 6542.49008175271


In [ ]:
# Average allocation per shelter

avg_total_alloc_greedy = alloc_men_avg_greedy + alloc_women_avg_greedy
avg_alloc_by_shelter_greedy = avg_total_alloc_greedy.sum(axis=1)

avg_total_alloc_random = alloc_men_avg_random + alloc_women_avg_random
avg_alloc_by_shelter_random = avg_total_alloc_random.sum(axis=1)

avg_alloc_df = pd.DataFrame({
    "Shelter": unique_shelters["LOCATION_NAME"].values,
    "Avg Alloc (Greedy)": avg_alloc_by_shelter_greedy,
    "Avg Alloc (Random)": avg_alloc_by_shelter_random
})

total_num_ppl = 8000
ppl_per_hotspot = np.round(top_16_hotspots["low_income_score"].values.reshape(-1, 1) * np.array([[0.5 * total_num_ppl, 0.5 * total_num_ppl]]), 0)

total_men_demand = ppl_per_hotspot[:, 0].sum()
total_women_demand = ppl_per_hotspot[:, 1].sum()

total_men_assigned_greedy = alloc_men_avg_greedy.sum()
total_women_assigned_greedy = alloc_women_avg_greedy.sum()

total_men_assigned_random = alloc_men_avg_random.sum()
total_women_assigned_random = alloc_women_avg_random.sum()

unsheltered_men_greedy = total_men_demand - total_men_assigned_greedy
unsheltered_women_greedy = total_women_demand - total_women_assigned_greedy

unsheltered_men_random = total_men_demand - total_men_assigned_random
unsheltered_women_random = total_women_demand - total_women_assigned_random

# Average total distance
total_distance_men_greedy = dist_men_avg_greedy
total_distance_women_greedy = dist_women_avg_greedy

total_distance_men_random = dist_men_avg_random
total_distance_women_random = dist_women_avg_random

# Avg distance per person
avg_dist_per_man_greedy = (
    total_distance_men_greedy / total_men_assigned_greedy
    if total_men_assigned_greedy > 0 else None
)

avg_dist_per_woman_greedy = (
    total_distance_women_greedy / total_women_assigned_greedy
    if total_women_assigned_greedy > 0 else None
)

avg_dist_per_man_random = (
    total_distance_men_random / total_men_assigned_random
    if total_men_assigned_random > 0 else None
)

avg_dist_per_woman_random = (
    total_distance_women_random / total_women_assigned_random
    if total_women_assigned_random > 0 else None
)

# PRINT REPORT
print("\n================ SUMMARY REPORT ================\n")

print("1) Average # allocated to each shelter:")
print(avg_alloc_df)

print("\n2) Total unsheltered:")
print(f"   Greedy: Men={unsheltered_men_greedy:.2f}, Women={unsheltered_women_greedy:.2f}")
print(f"   Random: Men={unsheltered_men_random:.2f}, Women={unsheltered_women_random:.2f}")

print("\n3) Total distance traveled:")
print(f"   Greedy: Men={total_distance_men_greedy:.2f}, Women={total_distance_women_greedy:.2f}")
print(f"   Random: Men={total_distance_men_random:.2f}, Women={total_distance_women_random:.2f}")

print("\n4) Average distance per person:")
print(f"   Greedy: Men={avg_dist_per_man_greedy:.4f}, Women={avg_dist_per_woman_greedy:.4f}")
print(f"   Random: Men={avg_dist_per_man_random:.4f}, Women={avg_dist_per_woman_random:.4f}")



================ SUMMARY REPORT ================

1) Average # allocated to each shelter:
                                              Shelter  Avg Alloc (Greedy)  \
0                                           Na-Me-Res                54.0   
1                              TSSS Womens' Residence                87.0   
2                                   TSSS Seaton House                66.0   
3                                TSSS Downsview Dells                18.0   
4                            TSSS Fort York Residence                68.0   
5                                            Sagatay                 17.0   
6             The Scott Mission - Kensington location                71.0   
7                              Dixon Hall Schoolhouse                30.0   
8                  Good Shepherd Centre Barrett House                 5.0   
9                                Good Shepherd Centre                25.0   
10                               Good Shepherd Centre         

### Run for 12 months

In [ ]:
simulation_df = pd.read_csv(r'demand_per_month_by_gender.csv', index_col=0)
simulation_df.head()

total_vs_male_ratio = simulation_df[['true_total', 'ratio_male']].copy()
total_vs_male_ratio

for idx, row in total_vs_male_ratio.iterrows():
    total_men = row['true_total']       # replace with your actual column name
    men_ratio = row['ratio_male']       # replace with your actual column name
    print(total_men)
    print(men_ratio)
    break

8051.0
0.6799126041239929


In [ ]:
for idx, row in total_vs_male_ratio.iterrows():
    total_ppl = row['true_total']
    men_ratio = row['ratio_male']
    
    # Allocation and distance for this simulation
    allo_men, allo_women, dist_men, dist_women = run_allocation_simulation(distance_matrix, top_16_hotspots, unique_shelters,total_num_ppl=total_ppl, men_pct=men_ratio, policy_type='greedy')
    
    # Average allocation per shelter
    avg_total_alloc = allo_men + allo_women
    avg_alloc_by_shelter = avg_total_alloc.sum(axis=1)
    
    avg_alloc_df = pd.DataFrame({
        "Shelter": unique_shelters["LOCATION_NAME"].values,
        "Avg Alloc": avg_alloc_by_shelter
    })
    
    # Total demand + unsheltered
    total_men_demand = total_ppl * men_ratio
    total_women_demand = total_ppl * (1 - men_ratio)
    
    total_men_assigned = allo_men.sum()
    total_women_assigned = allo_women.sum()
    
    unsheltered_men = total_men_demand - total_men_assigned
    unsheltered_women = total_women_demand - total_women_assigned
    
    # Total distance
    total_distance_men = dist_men
    total_distance_women = dist_women
    
    # Average distance per person
    avg_dist_per_man = total_distance_men / total_men_assigned if total_men_assigned > 0 else None
    avg_dist_per_woman = total_distance_women / total_women_assigned if total_women_assigned > 0 else None
    
    # Print report
    print(f"\n======= SIMULATION {idx} =======")
    print(f"Total population: {total_ppl}, Male ratio: {men_ratio:.2f}")
    
    print("\n1) Number allocated to each shelter:")
    print(avg_alloc_df)
    
    print("\n2) Total unsheltered:")
    print(f"   Men:   {unsheltered_men:.2f}")
    print(f"   Women: {unsheltered_women:.2f}")
    
    print("\n3) Total distance traveled:")
    print(f"   Men:   {total_distance_men:.2f}")
    print(f"   Women: {total_distance_women:.2f}")
    
    print("\n4) Distance per person:")
    print(f"   Men:   {avg_dist_per_man:.4f}")
    print(f"   Women: {avg_dist_per_woman:.4f}")


======= SIMULATION 1 =======
Total population: 8051.0, Male ratio: 0.68

1) Number allocated to each shelter:
                                              Shelter  Avg Alloc
0                                           Na-Me-Res       54.0
1                              TSSS Womens' Residence       87.0
2                                   TSSS Seaton House       66.0
3                                TSSS Downsview Dells       18.0
4                            TSSS Fort York Residence       68.0
5                                            Sagatay        17.0
6             The Scott Mission - Kensington location       71.0
7                              Dixon Hall Schoolhouse       30.0
8                  Good Shepherd Centre Barrett House        5.0
9                                Good Shepherd Centre       25.0
10                               Good Shepherd Centre       35.0
11                                  Cornerstone Place       28.0
12                   CONC Men's Shelter Lans

In [ ]:
for idx, row in total_vs_male_ratio.iterrows():
    total_ppl = row['true_total']
    men_ratio = row['ratio_male']
    
    # Allocation and distance for this simulation
    allo_men, allo_women, dist_men, dist_women = run_allocation_simulation(
        distance_matrix, top_16_hotspots, unique_shelters,
        total_num_ppl=total_ppl, men_pct=men_ratio, policy_type='random'
    )
    
    # Average allocation per shelter
    avg_total_alloc = allo_men + allo_women
    avg_alloc_by_shelter = avg_total_alloc.sum(axis=1)
    
    avg_alloc_df = pd.DataFrame({
        "Shelter": unique_shelters["LOCATION_NAME"].values,
        "Avg Alloc": avg_alloc_by_shelter
    })
    
    # Total demand + unsheltered
    total_men_demand = total_ppl * men_ratio
    total_women_demand = total_ppl * (1 - men_ratio)
    
    total_men_assigned = allo_men.sum()
    total_women_assigned = allo_women.sum()
    
    unsheltered_men = total_men_demand - total_men_assigned
    unsheltered_women = total_women_demand - total_women_assigned
    
    # Total distance
    total_distance_men = dist_men
    total_distance_women = dist_women
    
    # Average distance per person
    avg_dist_per_man = total_distance_men / total_men_assigned if total_men_assigned > 0 else None
    avg_dist_per_woman = total_distance_women / total_women_assigned if total_women_assigned > 0 else None
    
    # Print report
    print(f"\n======= SIMULATION {idx} =======")
    print(f"Total population: {total_ppl}, Male ratio: {men_ratio:.2f}")
    
    print("\n1) Number allocated to each shelter:")
    print(avg_alloc_df)
    
    print("\n2) Total unsheltered:")
    print(f"   Men:   {unsheltered_men:.2f}")
    print(f"   Women: {unsheltered_women:.2f}")
    
    print("\n3) Total distance traveled:")
    print(f"   Men:   {total_distance_men:.2f}")
    print(f"   Women: {total_distance_women:.2f}")
    
    print("\n4) Distance per person:")
    print(f"   Men:   {avg_dist_per_man:.4f}")
    print(f"   Women: {avg_dist_per_woman:.4f}")


======= SIMULATION 1 =======
Total population: 8051.0, Male ratio: 0.68

1) Number allocated to each shelter:
                                              Shelter  Avg Alloc
0                                           Na-Me-Res       54.0
1                              TSSS Womens' Residence       87.0
2                                   TSSS Seaton House       66.0
3                                TSSS Downsview Dells       18.0
4                            TSSS Fort York Residence       68.0
5                                            Sagatay        17.0
6             The Scott Mission - Kensington location       71.0
7                              Dixon Hall Schoolhouse       30.0
8                  Good Shepherd Centre Barrett House        5.0
9                                Good Shepherd Centre       25.0
10                               Good Shepherd Centre       35.0
11                                  Cornerstone Place       28.0
12                   CONC Men's Shelter Lans